# 1. explain note book

all data set  indexed by id_patiant and time per hour:

    1- all_df.pickle: X_data whithout processing
    
    2- all_original labels.pickle: Y_data also whithout preprocessing

the beginer observations from each file as past time (80%)represent the train set data 

and the last observations  as future time represent the test sey data (20%)

1) train set data indexed by id_patiant and per hour time: 

    1-1 train_set.pickle: X_train whithout processing
    
    1-2 train_original labels.pickle: Y_train also whithout preprocessing
    
2) test set data indexed by id_patiant and per hour time:

    2-1- test_set.pickle: X_test whithout processing
    2-2- test_original labels.pickle: Y_test also whithout preprocessing
 

# 2. libreries  import

In [399]:
import numpy as np, os, os.path, sys
import dill, pickle
from multiprocessing import Pool, cpu_count
from joblib import Parallel, delayed
import json
from sty import fg, bg
import pandas as pd
from math import floor
import shutil
import fidle

# 3. parametres

In [400]:
ROOT_DIR = os.path.dirname(os.path.abspath('vedio RNN'))

#----------------------------------------------------inputs---------------------------------
# Setup paths

DATA_DIR = ROOT_DIR + '/data/test/1000AB'
input_directory = 'C:/Users/hadjali/1 dataset/training_setAB_1000' #500/500 patient from  hospitals A/B

#DATA_DIR = ROOT_DIR + '/data/test/4AB'
#input_directory = 'C:/Users/hadjali/1 dataset/training_set_AB_4' #2/2 patient from  hospitals A/B

#DATA_DIR = ROOT_DIR + '/data/test/500A'
#input_directory='C:/Users/hadjali/1 dataset/training_set_A_500' #500 from hospital A

#DATA_DIR = ROOT_DIR + '/data/test/500B'
#input_directory='C:/Users/hadjali/1 dataset/training_set_B_500' #500 from hospital B

#DATA_DIR = ROOT_DIR + '/data/test/40336AB'
#input_directory='C:/Users/hadjali/1 dataset/training_set_AB_40336' #40336 from hospital B and B

#DATA_DIR = ROOT_DIR + '/data/test/20AB'
#input_directory='C:/Users/hadjali/1 dataset/training_set_AB_20' #20 from hospital B and B

#DATA_DIR = ROOT_DIR + '/data/test/20336A'
#input_directory = 'C:/Users/hadjali/1 dataset/training_set_A_20336' # 20336 patient from  hospitals A

#DATA_DIR = ROOT_DIR + '/data/test/20000B'
#input_directory = 'C:/Users/hadjali/1 dataset/training_set_B_20000' # 20000 patient from  hospitals B

#DATA_DIR = ROOT_DIR + '/data/test/p100005 B'
#input_directory='C:/Users/hadjali/1 dataset/p100005 B' # a nonseptic patient from hospital B

#DATA_DIR = ROOT_DIR + '/data/test/p000009 A'
#input_directory='C:/Users/hadjali/1 dataset/p000009 A' #a septic patient from hospital A

scale       =1 #pourcentage of dataset to be used (1=all dataset)

# if using future/past split method
#train_prop  =.8 #poucentage for train (the rest being for the test)

# if using paire/impaire split method
train_prop  =80 #poucentage for train (the rest being for the test)
val_prop    =70 #poucentage for subset train (the rest being for the validation set)

#---------------------------------results--------------------------------------------
#DATA_DIR + '/mean'
#for data,label,df_value in [('all_df',labels,df_values), ('train_set',labels_train,train_values), ('test_set',labels_test,test_values)]:
    #save_pickle(label, DATA_DIR + '/from_raw/Y/'+ data + '_original_labels.pickle')
    #save_pickle(df_value, DATA_DIR + '/from_raw/X/'+ data + '_raw.pickle')
    #x_y=pd.concat([df_values,labels],axis=1)
    #save_pickle(x_y, DATA_DIR + '/from_raw/XY/'+ data + '_x_y_raw.pickle')

# 4. functions

In [401]:
def save_pickle(obj, filename, use_dill=False, protocol=4):
    """ Basic pickle/dill dumping """
    create_folder_if_not_exist(filename)
    with open(filename, 'wb') as file:
        if not use_dill:
            pickle.dump(obj, file, protocol=protocol)
        else:
            dill.dump(obj, file)


def load_pickle(filename, use_dill=False):
    """ Basic pickle/dill loading function """
    with open(filename, 'rb') as file:
        if not use_dill:
            obj = pickle.load(file)
        else:
            obj = dill.load(file)
    return obj


def create_folder_if_not_exist(filename):
    #Makes a folder if the path does not already exist
    os.makedirs(os.path.dirname(filename), exist_ok=True)

def psv_to_dataframe(fname,scale=1):
    """ Transforms a single psv file into a dataframe with an id, ICULOS columns"""
    df = pd.read_csv(fname, sep='|')
    df['id'] = (fname.split('.psv')[0].split('/')[-1])
    df['patient'] = (fname.split('.psv')[0].split('/')[-1])
    df['patient1'] = (fname.split('.psv')[0].split('/')[-1])
    df['time'] = df['ICULOS']
    # Idx according to id and time
    df.set_index(['id','time'],inplace=True)
    df=df[:int(scale*len(df))]
    return df


def validation_test(train,val_prop=70):
    
    #train=load_pickle( DATA_DIR + '/from_raw/XY/train_set_x_y_raw.pickle')
    all_train_set=train
    train=train[:]
    patients_training_data = all_train_set['patient'].unique()
    #print(len(patients_training_data))
    #np.random.shuffle(patients_training_data) # melanger les patients
    from math import floor
    i=floor((val_prop*patients_training_data.shape[0])//100) # chois d'un entier  comme 80% de train
    #print(i)
    patients_training_data = patients_training_data[0:i]
    sub_train = train[train['patient'].isin(patients_training_data)]#.drop('patient', axis=1)
    val = train[~train['patient'].isin(patients_training_data)]#.drop('patient', axis=1)
    #subset_train=train
    return sub_train,val
        
def raw_data_transformation_to_raw_values_and_labels(location):
    
    """
    Given either a location of psv files or single psv file, transforms into dataframes indexed with id and time ICULOS

    :param location: either a folder containing psv files or a single psv file
    :return: df or (df,dataset_train,dataset_test) of all psv files indexed by [id, ICULOS]
    """

    if isinstance(location, list):
        fnames = [l + '/' + x for l in location for x in os.listdir(l)]
    elif not location.endswith('.psv'):
        fnames = [location + '/' + x for x in os.listdir(location)]
    else:
        fnames = [location]
    #----------------------------------
    is_septic_list=[]
    non_septic_list=[]
    for fname in fnames:
        df= psv_to_dataframe(fname)
        if np.any(df['SepsisLabel']):
            is_septic_list.append(df)
        else:
            non_septic_list.append(df)
            
    
    #----------------------------------

    if(len(fnames)==1):
        df= psv_to_dataframe(fnames[0])
        sub_train_set,val_set=train_set=test_set=df
    else:
        i=floor((train_prop*len(fnames))/100) # chois d'un entier  comme 80% de train
        #print(i)
        j=0

        list_df,list_train_set,list_test_set=[],[],[]
        for fname in fnames:
            df= psv_to_dataframe(fname)
            list_df.append(df)
            #if np.any(df['SepsisLabel']):
                #is_septic=True

            if (j<i):
                list_train_set.append(df)
            else:
                list_test_set.append(df)
            j=j+1
            """list_train_set.append(train_set)
            list_test_set.append(test_set)"""

        df = pd.concat(list_df)
        train_set = pd.concat(list_train_set)
        test_set = pd.concat(list_test_set)
        if(len(train_set['patient'].unique())==1):
            sub_train_set=val_set=train_set
        else:
            sub_train_set,val_set=validation_test(train_set,val_prop=70)
        
    # Get values and labels
    if 'SepsisLabel' in df.columns:
        df_values, labels = df.drop('SepsisLabel', axis=1), df['SepsisLabel']
    else:
        df_values = df

    if 'SepsisLabel' in train_set.columns:
        train_values, labels_train = train_set.drop('SepsisLabel', axis=1), train_set['SepsisLabel']
    else:
        train_values = train_set

    if 'SepsisLabel' in test_set.columns:
        test_values, labels_test = test_set.drop('SepsisLabel', axis=1), test_set['SepsisLabel']
    else:
        test_values = test_set
    
    if 'SepsisLabel' in sub_train_set.columns:
        sub_train_values, sub_labels_train = sub_train_set.drop('SepsisLabel', axis=1), sub_train_set['SepsisLabel']
    else:
        sub_train_values = sub_train_set
    
    if 'SepsisLabel' in val_set.columns:
        val_values, labels_val = val_set.drop('SepsisLabel', axis=1), val_set['SepsisLabel']
    else:
        val_values = val_set
        
    #---------------------calcule mean std from train set to be used for standrdiztion of data
    df_mean_std = train_values.describe().loc[['mean', 'std']]
    
    #save_pickle(df_mean_std, DATA_DIR + '/mean/mean_std_scaling_'+f'{train_values.shape[0]}'+'.pickle')
    save_pickle(df_mean_std, DATA_DIR + '/mean/mean_std.pickle')
    
    #----------------------------------------------------------------------------------------
    for data,label,df_value in [('all_df',labels,df_values), ('train_set',labels_train,train_values),
                                ('test_set',labels_test,test_values), ('sub_train_set',sub_labels_train,sub_train_values),
                               ('val_set',labels_val,val_values)]:
        
        save_pickle(label, DATA_DIR + '/from_raw/Y/'+ data + '_original_labels.pickle')
        save_pickle(df_value, DATA_DIR + '/from_raw/X/'+ data + '_raw.pickle')
        
        x_y=pd.concat([df_value,label],axis=1)
        save_pickle(x_y, DATA_DIR + '/from_raw/XY/'+ data + '_x_y_raw.pickle')
    #return df, df_values, labels,  train_set,train_values, labels_train,  test_set,test_values, labels_test
    return None

In [402]:
def raw_data_transformation_to_raw_values_and_labels_is_is_not_septic(is_is_not_list):

    if(len(is_is_not_list)==1):
        sub_train_set=val_set=train_set=test_set=is_is_not_list[0]
    else:
        #if(is_is_not_list=True)
        i=floor((80*len(is_is_not_list))//100) # chois d'un entier  comme 80% de train
        #print(i)
        j=0

        list_train_set,list_test_set=[],[]
        for is_is_not in is_is_not_list:
            #list_df.append(df)
            if (j<i):
                list_train_set.append(is_is_not)
            else:
                list_test_set.append(is_is_not)
            j=j+1
            """list_train_set.append(train_set)
            list_test_set.append(test_set)"""

        #df = pd.concat(list_df)
        train_set = pd.concat(list_train_set)
        test_set = pd.concat(list_test_set)

        """if(len(train_set['patient'].unique())==1):
            sub_train_set=val_set=train_set
        else:"""
    sub_train_set,val_set=validation_test(train_set,val_prop=70)
        
        

    return train_set, test_set, sub_train_set,val_set

In [403]:
def raw_data_transformation_to_raw_values_and_labels1(location):
    
    """
    Given either a location of psv files or single psv file, transforms into dataframes indexed with id and time ICULOS

    :param location: either a folder containing psv files or a single psv file
    :return: df or (df,dataset_train,dataset_test) of all psv files indexed by [id, ICULOS]
    """

    if isinstance(location, list):
        fnames = [l + '/' + x for l in location for x in os.listdir(l)]
    elif not location.endswith('.psv'):
        fnames = [location + '/' + x for x in os.listdir(location)]
    else:
        fnames = [location]
    #----------------------------------
    list_df=[]
    
    if(len(fnames)==1):
        df1= psv_to_dataframe(fnames[0])
        list_df.append(df1)
        df = pd.concat(list_df)
        sub_train_set=val_set=train_set=test_set=df
    else:
        
        
        is_septic_list=[]
        non_septic_list=[]
        for fname in fnames:
            df1= psv_to_dataframe(fname)
            list_df.append(df1)
            if np.any(df1['SepsisLabel']):
                is_septic_list.append(df1)
            else:
                non_septic_list.append(df1)

        print ('number of positif patient  ',len(is_septic_list))
        print ('number of negatif patient  ',len(non_septic_list))
        #df = pd.concat(list_df)
        is_train_set, is_test_set, is_sub_train_set,is_val_set=raw_data_transformation_to_raw_values_and_labels_is_is_not_septic(is_septic_list)        
        is_not_train_set, is_not_test_set, is_not_sub_train_set,is_not_val_set=raw_data_transformation_to_raw_values_and_labels_is_is_not_septic(non_septic_list)
        #----------------------------------

        df = pd.concat(list(list_df))
        #print(df.shape)

        #train_set = pd.concat(is_train_set)
        #no_train_set = pd.concat(is_not_train_set)
        train_set = pd.concat([is_train_set,is_not_train_set], axis=0)

        test_set = pd.concat([is_test_set,is_not_test_set],axis=0)
        #test_set = pd.concat(is_not_test_set)

        sub_train_set = pd.concat([is_sub_train_set, is_not_sub_train_set],axis=0)
        #sub_train_set = pd.concat(is_not_sub_train_set)

        val_set = pd.concat([is_val_set, is_not_val_set], axis=0)
        #val_set = pd.concat(is_not_val_set)

    # Get values and labels
    if 'SepsisLabel' in df.columns:
        df_values, labels = df.drop('SepsisLabel', axis=1), df['SepsisLabel']
    else:
        df_values = df

    if 'SepsisLabel' in train_set.columns:
        train_values, labels_train = train_set.drop('SepsisLabel', axis=1), train_set['SepsisLabel']
    else:
        train_values = train_set

    if 'SepsisLabel' in test_set.columns:
        test_values, labels_test = test_set.drop('SepsisLabel', axis=1), test_set['SepsisLabel']
    else:
        test_values = test_set
    
    if 'SepsisLabel' in sub_train_set.columns:
        sub_train_values, sub_labels_train = sub_train_set.drop('SepsisLabel', axis=1), sub_train_set['SepsisLabel']
    else:
        sub_train_values = sub_train_set
    
    if 'SepsisLabel' in val_set.columns:
        val_values, labels_val = val_set.drop('SepsisLabel', axis=1), val_set['SepsisLabel']
    else:
        val_values = val_set
        
    #---------------------calcule mean std from train set to be used for standrdiztion of data
    df_mean_std = train_values.describe().loc[['mean', 'std']]
    
    #save_pickle(df_mean_std, DATA_DIR + '/mean/mean_std_scaling_'+f'{train_values.shape[0]}'+'.pickle')
    save_pickle(df_mean_std, DATA_DIR + '/mean/mean_std.pickle')
    
    #----------------------------------------------------------------------------------------
    for data,label,df_value in [('all_df',labels,df_values), ('train_set',labels_train,train_values),
                                ('test_set',labels_test,test_values), ('sub_train_set',sub_labels_train,sub_train_values),
                               ('val_set',labels_val,val_values)]:
        
        save_pickle(label, DATA_DIR + '/from_raw/Y/'+ data + '_original_labels.pickle')
        save_pickle(df_value, DATA_DIR + '/from_raw/X/'+ data + '_raw.pickle')
        
        x_y=pd.concat([df_value,label],axis=1)
        save_pickle(x_y, DATA_DIR + '/from_raw/XY/'+ data + '_x_y_raw.pickle')
    #return df, df_values, labels,  train_set,train_values, labels_train,  test_set,test_values, labels_test
    return None

# 5. main call

In [404]:
#df,df_values,labels,  train_set,train_values,labels_train,  test_set,test_values,labels_test =raw_data_transformation_to_raw_values_and_labels(input_directory)
raw_data_transformation_to_raw_values_and_labels1(input_directory)

number of positif patient   71
number of negatif patient   929
56
56
39
743
743
520
(38890, 43)


# 6. few statistics

In [417]:
for data,label,df_value in [('all_df','all labels','all df_values'), ('train_set','labels_train','train_values'), 
                            ('test_set','labels_test','test_values'),
                           ('sub_train_set','sub_labels_train','sub_train_values'), ('val_set','labels_val','val_values') ]:
    fidle.utils.subtitle('-------------' + f'{data}'+'   ----------------')
    
    #labels=load_pickle( DATA_DIR + '/from_raw/Y/'+ data + '_original_labels.pickle')
    #print(f'{label}'+ '  shape y ',labels.shape)
    
    #x_values=load_pickle( DATA_DIR + '/from_raw/X/'+ data + '_raw.pickle')
    #print(f'{df_value}'+ ' shape x ',x_values.shape)
    #display(f'{df_value}'+' display  x ',x_values.head())
        
    x_y_values=load_pickle( DATA_DIR + '/from_raw/XY/'+ data + '_x_y_raw.pickle')
    
    if(data=='all_df'):
        all_size=x_y_values.shape[0]
        all_patient=len(x_y_values['patient'].unique())
        print('pourcentage **** all df*************   ', '100' +'  %')
        print('pourcentage **** all patient*************   ', '100' +'  %')
        print("\n")
        train_x_y_values=load_pickle( DATA_DIR + '/from_raw/XY/train_set_x_y_raw.pickle')
        #else:
        #if(data=='train_set'):
        train_size=train_x_y_values.shape[0]
        train_patient=len(train_x_y_values['patient'].unique())
        print('pourcentage **** train_set*************   ',train_x_y_values.shape[0]* 100/all_size )
        #print('pourcentage **** train patients *************   ',train_patient* 100/all_patient )
        
        test_x_y_values=load_pickle( DATA_DIR + '/from_raw/XY/test_set_x_y_raw.pickle')
        #else:
        #if(data=='test_set'):
        test_size=test_x_y_values.shape[0]
        test_patient=len(test_x_y_values['patient'].unique())
        print('pourcentage *** test_set*************   ',test_x_y_values.shape[0]* 100/all_size )
        print("\n")
        print('pourcentage **** train patients *************   ',train_patient* 100/all_patient )
        print('pourcentage **** test patients *************   ',test_patient* 100/all_patient )
        
        #else:
    if(data=='train_set'):

        sub_train_x_y_values=load_pickle( DATA_DIR + '/from_raw/XY/sub_train_set_x_y_raw.pickle')
        #if(data=='train_set'):
        train_size=x_y_values.shape[0]
        train_patient=len(x_y_values['patient'].unique())

        sub_train_set_size=sub_train_x_y_values.shape[0]
        sub_train_patient=len(sub_train_x_y_values['patient'].unique())
        print('pourcentage *** sub_train_set*************   ',sub_train_x_y_values.shape[0]* 100/train_size )
        #print('pourcentage **** sub_train patients *************   ',sub_train_patient* 100/train_patient )

        #else:
        #if(data=='val_set'):
        val_train_x_y_values=load_pickle( DATA_DIR + '/from_raw/XY/val_set_x_y_raw.pickle')
        val_set_size=val_train_x_y_values.shape[0]
        val_patient=len(val_train_x_y_values['patient'].unique())
        print('pourcentage *** val_set *************   ',val_train_x_y_values.shape[0]* 100/train_size )
        print("\n")
        print('pourcentage **** sub_train patients *************   ',sub_train_patient* 100/train_patient )
        print('pourcentage **** val patients *************   ',val_patient* 100/train_patient )

    print("--------------")
    print('number of all patients  '+data +'     ',len(x_y_values['patient'].unique()))
    is_septic_list=0
    non_septic_list=0
    is_septic=0
    non_septic=0
    group=x_y_values.groupby(['patient'])
    for i,g in group:
        #df1= psv_to_dataframe(fname)
        #list_df.append(df1)
        if np.any(g['SepsisLabel']):
            is_septic +=1 
            is_septic_list += g.shape[0]
        else:
            non_septic +=1
            non_septic_list += g.shape[0]
    print ('number of positif patient  ',is_septic)
    print ('number of negatif patient  ',non_septic)
    print("\n")
    print ('number of all observations for positif patients   ',is_septic_list)
    positive_patient_list=x_y_values['patient'][x_y_values['SepsisLabel']==1.0].unique()
    positive_patient_df=x_y_values[(x_y_values.patient.isin(positive_patient_list))]
    
    #print('number of positives observations/ number of negatives observations for positif patients  '+data +'  \n',positive_patient_df['SepsisLabel'].value_counts())
    print('where :::::::::  '+data +'  \n',positive_patient_df['SepsisLabel'].value_counts())
    print("\n")
    print ('number of all observations for negatif paients   ',non_septic_list)
    #list of negative  patients
    negative_patient_list=x_y_values['patient'][x_y_values['SepsisLabel']==0.0].unique()
    # construct dataframe of negative patients
    negative_patient_df=x_y_values[x_y_values.patient.isin(negative_patient_list)]
    #print('negative_patient_list  '+data +'     ',len(negative_patient_list))
    
    #list of positive patients
    
    # construct dataframe of positive patients
    #positive_patient_df=x_y_values[(x_y_values.patient.isin(positive_patient_list))]
    #positive_patient_df = x_y_values[~(x_y_values['patient'].isin(negative_patient_list))]
    #print('positive_patient_list  '+data +'     ',len(positive_patient_list))
    print("--------------")
    print('all observations  '+data +'     ',x_y_values.shape[0])
    
    #print('negatiffffffffffffffffffffff observations  '+data +'   ',negative_patient_df['SepsisLabel'].value_counts())
    #print('negatives and positives observations  '+data +'  \n',x_y_values['SepsisLabel'].value_counts())
    print('where :::  '+data +'  \n',x_y_values['SepsisLabel'].value_counts())
    print("--------------")
    
#mean=load_pickle(DATA_DIR + '/mean/mean_std.pickle')
#display('mean  '+' display  x ',mean.head())

<br>**-------------all_df   ----------------**

pourcentage **** all df*************    100  %
pourcentage **** all patient*************    100  %


pourcentage **** train_set*************    79.28259192594497
pourcentage *** test_set*************    20.717408074055026


pourcentage **** train patients *************    79.9
pourcentage **** test patients *************    20.1
--------------
number of all patients  all_df      1000
number of positif patient   71
number of negatif patient   929


number of all observations for positif patients    4703
where :::::::::  all_df  
 0    4029
1     674
Name: SepsisLabel, dtype: int64


number of all observations for negatif paients    34187
--------------
all observations  all_df      38890
where :::  all_df  
 0    38216
1      674
Name: SepsisLabel, dtype: int64
--------------


<br>**-------------train_set   ----------------**

pourcentage *** sub_train_set*************    69.15966659099017
pourcentage *** val_set *************    30.840333409009826


pourcentage **** sub_train patients *************    69.96245306633291
pourcentage **** val patients *************    30.037546933667084
--------------
number of all patients  train_set      799
number of positif patient   56
number of negatif patient   743


number of all observations for positif patients    3456
where :::::::::  train_set  
 0    2923
1     533
Name: SepsisLabel, dtype: int64


number of all observations for negatif paients    27377
--------------
all observations  train_set      30833
where :::  train_set  
 0    30300
1      533
Name: SepsisLabel, dtype: int64
--------------


<br>**-------------test_set   ----------------**

--------------
number of all patients  test_set      201
number of positif patient   15
number of negatif patient   186


number of all observations for positif patients    1247
where :::::::::  test_set  
 0    1106
1     141
Name: SepsisLabel, dtype: int64


number of all observations for negatif paients    6810
--------------
all observations  test_set      8057
where :::  test_set  
 0    7916
1     141
Name: SepsisLabel, dtype: int64
--------------


<br>**-------------sub_train_set   ----------------**

--------------
number of all patients  sub_train_set      559
number of positif patient   39
number of negatif patient   520


number of all observations for positif patients    2284
where :::::::::  sub_train_set  
 0    1909
1     375
Name: SepsisLabel, dtype: int64


number of all observations for negatif paients    19040
--------------
all observations  sub_train_set      21324
where :::  sub_train_set  
 0    20949
1      375
Name: SepsisLabel, dtype: int64
--------------


<br>**-------------val_set   ----------------**

--------------
number of all patients  val_set      240
number of positif patient   17
number of negatif patient   223


number of all observations for positif patients    1172
where :::::::::  val_set  
 0    1014
1     158
Name: SepsisLabel, dtype: int64


number of all observations for negatif paients    8337
--------------
all observations  val_set      9509
where :::  val_set  
 0    9351
1     158
Name: SepsisLabel, dtype: int64
--------------
